Metti la HF Token  nei secret del colab
Serve per l'uso di modelli con req token




In [ ]:
# Standard library
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')
if HF_TOKEN:
    login(HF_TOKEN)
    print("Successfully logged in to Hugging Face!")
else:
    print("Token is not set. Please save the token first.")


In [ ]:
import os
import zipfile

# URL for the dataset
url = "https://zenodo.org/records/15700269/files/datasetWSI.zip?download=1"

# Download the file using wget
!wget -O /content/Train.zip "$url"

# Define the extraction path
extract_path = './'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile('./Train.zip', 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the contents of the extracted folder
extracted_files = os.listdir(extract_path)
print("Extracted files:", extracted_files)


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

!conda create -n "trident" python=3.10

In [ ]:
!git clone https://github.com/mahmoodlab/trident.git
!conda run -n trident pip install -e trident/.

In [ ]:
# Crea un archivio ZIP della cartella
import zipfile # Import the zipfile module

def create_zip_archive(folder_path, zip_name=None):
    """
    Crea un archivio ZIP di una cartella mantenendo la struttura
    """
    if zip_name is None:
        zip_name = f"{os.path.basename(folder_path)}.zip"

    print(f"🗜️ Creando archivio ZIP: {zip_name}")

    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Mantiene la struttura delle cartelle nell'archivio
                arc_name = os.path.relpath(file_path, os.path.dirname(folder_path))
                zipf.write(file_path, arc_name)
                print(f"  📁 Aggiunto: {arc_name}")

    # Mostra dimensione dell'archivio
    zip_size = os.path.getsize(zip_name)
    print(f"✅ Archivio creato: {zip_name} ({zip_size/1024/1024:.2f} MB)")
    return zip_name

In [ ]:
import torch
import requests
import json
import os

# Token di accesso Zenodo (sostituisci con il tuo)
ACCESS_TOKEN = 'uVSb7icJqT9efPM71KYgviJ50r7eML9ynei2q7hDkedVlFrf8fBsr9lFaJ3O'

# Crea una nuova deposizione
def create_deposition(title):
    url = 'https://zenodo.org/api/deposit/depositions'
    headers = {"Content-Type": "application/json"}
    params = {'access_token': ACCESS_TOKEN}

    data = {
        'metadata': {
            'title': title,
            'upload_type': 'dataset',
            'description': 'Dataset WSI project MLiA',
            'creators': [{'name': 'Raf-Tony-Luca'}]
        }
    }

    r = requests.post(url, params=params, data=json.dumps(data), headers=headers)
    return r.json()

# Carica il file
def upload_file(deposition_id, file_path):
    # Get bucket URL
    url = f'https://zenodo.org/api/deposit/depositions/{deposition_id}'
    params = {'access_token': ACCESS_TOKEN}
    r = requests.get(url, params=params)
    bucket_url = r.json()["links"]["bucket"]

    # Upload file
    filename = os.path.basename(file_path)
    with open(file_path, "rb") as fp:
        r = requests.put(f"{bucket_url}/{filename}",
                        data=fp,
                        params=params)
    return r.json()

# Pubblica il dataset
def publish_deposition(deposition_id):
    url = f'https://zenodo.org/api/deposit/depositions/{deposition_id}/actions/publish'
    params = {'access_token': ACCESS_TOKEN}
    r = requests.post(url, params=params)
    return r.json()


In [ ]:
import torch
import gc

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Libera la cache della GPU
        torch.cuda.ipc_collect()    # Raccoglie la memoria non più usata dalla GPU
    gc.collect()                    # Libera la RAM

free_memory()


In [ ]:
!MPLBACKEND=Agg conda run -n trident python trident/run_batch_of_slides.py --task all --wsi_dir ./ndpi_files/B --job_dir ./trident_processed_univ2/B --patch_encoder uni_v2 --patch_size 256 --mag 20

In [ ]:
import torch
import gc

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Libera la cache della GPU
        torch.cuda.ipc_collect()    # Raccoglie la memoria non più usata dalla GPU
    gc.collect()                    # Libera la RAM

free_memory()


In [ ]:
!MPLBACKEND=Agg conda run -n trident python trident/run_batch_of_slides.py --task all --wsi_dir ./ndpi_files/E --job_dir ./trident_processed_univ2/E --patch_encoder uni_v2 --patch_size 256 --mag 20

In [ ]:
import torch
import gc

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Libera la cache della GPU
        torch.cuda.ipc_collect()    # Raccoglie la memoria non più usata dalla GPU
    gc.collect()                    # Libera la RAM

free_memory()


In [ ]:
!MPLBACKEND=Agg conda run -n trident python trident/run_batch_of_slides.py --task all --wsi_dir ./ndpi_files/S --job_dir ./trident_processed_univ2/S --patch_encoder uni_v2 --patch_size 256 --mag 20

In [ ]:
import torch
import gc

def free_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()    # Libera la cache della GPU
        torch.cuda.ipc_collect()    # Raccoglie la memoria non più usata dalla GPU
    gc.collect()                    # Libera la RAM

free_memory()


In [ ]:
# Percorso della cartella da caricare
folder_to_upload = '/content/trident_processed_univ2'  # Cambia con il percorso della tua cartella
zip_filename = 'datasetTrident_univ2.zip'  # Nome dell'archivio
zip_path = create_zip_archive(folder_to_upload, zip_filename)


In [ ]:
# Esegui l'upload
print("Creando deposizione...")
deposition = create_deposition("dataset_trident_univ2")
deposition_id = deposition['id']

print(f"Caricando file... (ID: {deposition_id})")
upload_result = upload_file(deposition_id, zip_filename)

print("Pubblicando dataset...")
publication = publish_deposition(deposition_id)

print(f"Dataset pubblicato! DOI: {publication['doi']}")
print(f"URL: {publication['links']['record_html']}")